In [1]:
# 09/03/2022
# Francisco Domínguez Mateos
# PSR3D Project
# LSTM Alarm Prediction
# 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [7]:
def sliding_windows(data, seq_length):
    x = []
    y = []
    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)
    return np.array(x),np.array(y)

In [32]:
class PSR3D_LSTM_Alarm_Prediction(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(PSR3D_LSTM_Alarm_Prediction, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)) 
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        return out

In [26]:
num_epochs = 2000
learning_rate = 0.01

column_number=1700
alarm_number=1
input_size = column_number
hidden_size = 2
num_layers = 1

num_classes = column_number+alarm_number

In [27]:
data_length=10000
sequence_length=10

In [28]:
data_signals=np.random.rand(data_length,column_number)
print(data.shape)

(10000, 1700)


In [29]:
data_sequence_input_signals,data_sequence_next_signals=sliding_windows(data_signals,sequence_length)
print(data_sequence_input_signals.shape)
print(data_sequence_next_signals.shape)

(9989, 10, 1700)
(9989, 1700)


In [30]:
data_alarms=np.random.rand(data_sequence_output.shape[0],1)
data_sequence_output=np.concatenate((data_sequence_next_signals,data_alarms),axis=1)
print(data_alarms.shape)
print(data_sequence_output.shape)

(9989, 1)
(9989, 1701)


In [36]:
train_size = int(data_sequence_input_signals.shape[0] * 0.67)
test_size = len(data_sequence_input_signals) - train_size

dataX = Variable(torch.Tensor(np.array(data_sequence_input_signals)))
dataY = Variable(torch.Tensor(np.array(data_sequence_output)))

trainX = Variable(torch.Tensor(np.array(data_sequence_input_signals[0:train_size])))
trainY = Variable(torch.Tensor(np.array(data_sequence_output[0:train_size])))

testX = Variable(torch.Tensor(np.array(data_sequence_input_signals[train_size:])))
testY = Variable(torch.Tensor(np.array(data_sequence_output[train_size:])))
print(dataX.shape)
print(dataY.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

torch.Size([9989, 10, 1700])
torch.Size([9989, 1701])
torch.Size([6692, 10, 1700])
torch.Size([6692, 1701])
torch.Size([3297, 10, 1700])
torch.Size([3297, 1701])


In [37]:
lstm = PSR3D_LSTM_Alarm_Prediction(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    # Forward step
    outputs = lstm(trainX)
    # Optimizing - backward step
    optimizer.zero_grad()
    # obtain the loss function
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Epoch: 0, loss: 0.53420
Epoch: 100, loss: 0.10073
Epoch: 200, loss: 0.08356
Epoch: 300, loss: 0.08334
Epoch: 400, loss: 0.08334
Epoch: 500, loss: 0.08334
Epoch: 600, loss: 0.08334
Epoch: 700, loss: 0.08334
Epoch: 800, loss: 0.08334
Epoch: 900, loss: 0.08334
Epoch: 1000, loss: 0.08334
Epoch: 1100, loss: 0.08334
Epoch: 1200, loss: 0.08334
Epoch: 1300, loss: 0.08334
Epoch: 1400, loss: 0.08334
Epoch: 1500, loss: 0.08334
Epoch: 1600, loss: 0.08334
Epoch: 1700, loss: 0.08334
Epoch: 1800, loss: 0.08334
Epoch: 1900, loss: 0.08334


In [38]:
lstm.eval()
train_predict = lstm(dataX)

data_predict  = train_predict.data.numpy()
dataY_plot    = dataY.data.numpy()

data_predict  = sc.inverse_transform(data_predict)
dataY_plot    = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.suptitle('Time-Series Prediction')
plt.show()

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.